In [ ]:
from flask import Flask, render_template, request
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
import joblib

In [ ]:
from flask import Flask, render_template, request
import pandas as pd
import joblib

app = Flask(__name__)

# Load the ARIMA models for different time points
arima_model_t2 = joblib.load('arima_model_t2.pkl')
arima_model_t3 = joblib.load('arima_model_t3.pkl')
arima_model_t4 = joblib.load('arima_model_t4.pkl')

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # Get the uploaded file from the form
        uploaded_file = request.files['file']

        # Read the uploaded file into a pandas DataFrame
        df = pd.read_excel(uploaded_file)

        # Determine which ARIMA model to use based on the number of columns
        num_columns = df.shape[1]
        if num_columns == 12:
            arima_model = arima_model_t2
        elif num_columns == 19:
            arima_model = arima_model_t3
        elif num_columns == 26:
            arima_model = arima_model_t4
        else:
            return "Invalid number of columns in the uploaded file."

        X = df.iloc[:, :num_columns]
        X = X.values.reshape(-1, num_columns)
        y = X[:, 0]  # Assuming the first column is 'Gewicht_T1'

        # Get predictions using the selected ARIMA model
        predictions = arima_model.predict(start=len(y), end=len(y) + len(df) - 1, exog=X)

        # Return the predictions to the user
        return render_template('result.html', predictions=predictions)

    return render_template('upload.html')

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
